## Week 3 : Vector Search

## Q1. Getting the embeddings model

In [36]:
from elasticsearch import Elasticsearch
import numpy as np
import pandas as pd
import requests
from sentence_transformers import SentenceTransformer
from tqdm.auto import tqdm

In [37]:
embedding_model = SentenceTransformer("multi-qa-distilbert-cos-v1")

KeyboardInterrupt: 

In [ ]:
user_question = "I just discovered the course. Can I still join it?"

In [ ]:
vector_question = embedding_model.encode(user_question)

**What's the first value of the resulting vector?**

In [ ]:
vector_question[0]

*Answer: 0.07*

### Prepare the documents

In [ ]:
base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'

docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [ ]:
filtered_documents = [d for d in documents if d["course"] == "machine-learning-zoomcamp"]
len(filtered_documents)

## Q2. Creating the embeddings

In [ ]:
embeddings = []

for doc in filtered_documents:
    qa_text = f"{doc['question']} {doc['text']}"
    vector = embedding_model.encode(qa_text).tolist()
    embeddings.append(vector)

In [ ]:
X = np.array(embeddings)
X.shape

## Q3. Search

**What's the highest score in the results?**

In [ ]:
v = vector_question
scores = X.dot(v)

In [ ]:
scores.max()

*Answer:* 0.65

### Vector search

In [ ]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=documents, embeddings=X)
search_engine.search(v, num_results=5)

## Q4. Hit-rate for our search engine

In [ ]:
base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [ ]:
question_embedded = []

for i in range(0, len(ground_truth)):
    vector_question = embedding_model.encode(ground_truth[i]['question'])
    question_embedded.append(vector_question)

KeyboardInterrupt: 

In [ ]:
relevance_total = []

for q in tqdm(question_embedded):
    results = search_engine.search(q, num_results=5)
    relevance = [d['id'] == doc_id for d in results]
    relevance_total.append(relevance)

In [ ]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

**What did you get?**

In [ ]:
hit_rate(relevance_total)

*Answer:*

## Q5. Indexing with ElasticSearch

In [ ]:
es_client = Elasticsearch('http://localhost:9200') 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "question_vector": {
                "type": "dense_vector",
                "dims": 375,
                "index": True,
                "similarity": "cosine"
            },
            "text_vector": {
                "type": "dense_vector",
                "dims": 375,
                "index": True,
                "similarity": "cosine"
            },
            "question_text_vector": {
                "type": "dense_vector",
                "dims": 375,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

In [ ]:
for doc in tqdm(documents):
    question = doc['question']
    text = doc['text']
    qt = question + ' ' + text

    doc['question_vector'] = embedding_model.encode(question)
    doc['text_vector'] = embedding_model.encode(text)
    doc['question_text_vector'] = embedding_model.encode(qt)

In [ ]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

In [ ]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["text"],
                        "type": "best_fields"
                    }
                },
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [ ]:
v_q = embedding_model.encode(user_question)

**What's the ID of the document with the highest score?**

*Answer:*

## Q6. Hit-rate for Elasticsearch

In [ ]:
relevance_total = []

for q in tqdm(ground_truth):
    doc_id = q['document']
    results = elastic_search(query=q['question'])
    relevance = [d['id'] == doc_id for d in results]
    relevance_total.append(relevance)

In [ ]:
hit_rate(relevance_total)

**What's hitrate for our dataset for Elastic?**

*Answer:*